Data from:

Physician & Other Supplier Payments - Detailed Data
https://www.cms.gov/Research-Statistics-Data-and-Systems/Statistics-Trends-and-Reports/Medicare-Provider-Charge-Data/Physician-and-Other-Supplier2017

Hospital Outpatient - Detailed Data
https://www.cms.gov/Research-Statistics-Data-and-Systems/Statistics-Trends-and-Reports/Medicare-Provider-Charge-Data/Outpatient

APC to CPT/HCPCS crosswalk - Addendum B – January 2020
https://www.cms.gov/Medicare/Medicare-Fee-for-Service-Payment/HospitalOutpatientPPS/Addendum-A-and-Addendum-B-Updates

Zip Code to CBSA
https://www.huduser.gov/portal/datasets/usps_crosswalk.html![image.png](attachment:image.png)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
%matplotlib inline

In [3]:
pd.options.display.max_columns = 25

In [4]:
pd.options.display.max_rows = 25

In [5]:
# Reading in the CBSA to ZIP code crosswalk

cbsa_to_zip = pd.read_excel('../data/CBSA_ZIP_032020.xlsx', 
                            usecols = ['CBSA', 'ZIP'],
                            dtypes={'ZIP':'str'})

In [6]:
#Reading in HCPCS to APC crosswalk with Short Descriptor column

hcpcs_to_apc = pd.read_excel('../data/Addendum_B/2020_january_web_addendum_b.12312019.xlsx', header = 2, usecols = ['HCPCS Code', 'Short Descriptor', 'APC '])

In [7]:
#Reading in the Physician & Other Supplier Payments while discarding some columns

phys_other_payments = pd.read_csv('../data/Medicare_Provider_Util_Payment_PUF_CY2017/Medicare_Provider_Util_Payment_PUF_CY2017.txt', 
    sep='\t', 
    skiprows = [1], 
    usecols = lambda column : column not in ['average_submitted_chrg_amt', 'average_Medicare_payment_amt', 'average_Medicare_standard_amt', 'hcpcs_drug_indicator', 'medicare_participation_indicator'], 
    dtype = {'nppes_provider_zip':'str'},
    low_memory=False)

In [8]:
# Reading in the Hospital Outpatient while discarding some columns

column_exclude_list = ['Outlier\nComprehensive\nAPC\nServices', 'Average\nMedicare\nOutlier\nAmount', 'Average\nEstimated\nTotal\nSubmitted\nCharges', 'Average\nMedicare\nPayment\nAmount']

hosp_payments = pd.read_excel('../data/MUP_OHP_R19_P04_V10_D17_APC_Provider/MUP_OHP_R19_P04_V10_D17_APC_Provider.xlsx', 
                              header = 5, 
                              usecols = lambda column : column not in column_exclude_list)

In [9]:
hcpcs_to_apc.columns = ['hcpcs', 'descriptor', 'apc']

In [10]:
cbsa_to_zip.columns = ['cbsa', 'zip']

In [11]:
phys_other_payments.columns = ['npi', 'last_org_name', 'first_name', 'mi', 'creds', 'gender', 'entity', 'street1', 'street2' , 'city', 'zip','state', 'country', 'provider_type', 'service_loc', 'hcpcs', 'description', 'services_cnt', 'benefic_cnt', 'benefic_d_cnt', 'avg_medi_allowed_amt']

In [12]:
hosp_payments.columns = ['provider_id', 'provider_name', 'street', 'city', 'state', 'zip', 'region', 'apc', 'description', 'benefic_cnt', 'services_cnt', 'avg_medi_allowed_amt']

In [13]:
# Dropping non US

phys_other_payments = phys_other_payments[phys_other_payments.country == 'US']

In [14]:
# Making a dataframe with only office entities to compare with the Hospital Data.

phys_other_payments = phys_other_payments[phys_other_payments['entity'] == 'O']

In [15]:
# Making a dataframe with only Ambulatory Surgical Centers

phys_other_payments = phys_other_payments[phys_other_payments['provider_type'] == 'Ambulatory Surgical Center']

In [18]:
phys_other_payments['zip'] = phys_other_payments['zip'].str[:5]

In [21]:
# Putting leading zero back on zips

cbsa_to_zip['zip'] = cbsa_to_zip['zip'].apply(lambda x: '{0:0>5}'.format(x))

In [23]:
# Merging CBSA to Zip in physcian billing data

phys_asc_cbsa = pd.merge(phys_other_payments, cbsa_to_zip, how='left', on = ['zip'])

del phys_other_payments

In [25]:
# Dropping NaN values in the apc column of hcpcs to apc crosswalk

hcpcs_to_apc = hcpcs_to_apc.dropna(subset=['apc'])

In [28]:
# Putting leading zero back on zips

hosp_payments['zip'] = hosp_payments['zip'].apply(lambda x: '{0:0>5}'.format(x))

In [32]:
# Adding matching APC code to HCPCS in the office dataframe, keeping only rows that have an apc match

phys_asc_apc = pd.merge(phys_asc_cbsa, hcpcs_to_apc, how = 'inner', on = ['hcpcs'])

del phys_asc_cbsa

In [33]:
phys_apc_data = phys_asc_apc.drop(['first_name', 'mi', 'creds', 'gender'], axis = 1)

del phys_asc_apc

In [34]:
# Adding CBSA column to hospital data, keeping only rows with a CBSA

hosp_payments_cbsa = pd.merge(hosp_payments, cbsa_to_zip, how = 'inner', on = ['zip'])

del hosp_payments

In [35]:
cbsa_to_region = hosp_payments_cbsa.loc[:,['region', 'cbsa']]

In [36]:
cbsa_to_region = cbsa_to_region.drop_duplicates(subset=['cbsa'])

In [37]:
apc_to_description = hosp_payments_cbsa.loc[:, ['apc', 'description']]

In [38]:
apc_to_description = apc_to_description.drop_duplicates(subset=['apc'])

In [39]:
list = []

for cbsa in hosp_payments_cbsa['cbsa'].unique():
    for apc in hosp_payments_cbsa['apc'].unique():
        selection = hosp_payments_cbsa[(hosp_payments_cbsa['cbsa'] == cbsa) & (hosp_payments_cbsa['apc'] == apc)]
        max_value = round(selection.avg_medi_allowed_amt.max(), 2)
        avg_value = round(selection.avg_medi_allowed_amt.mean(), 2)
        min_value = round(selection.avg_medi_allowed_amt.min(), 2)
        list.append([cbsa, apc, max_value, avg_value, min_value])

In [40]:
list_df = pd.DataFrame(list)

del list

In [41]:
list_df.columns = ['cbsa','apc', 'hosp_max', 'hosp_avg', 'hosp_min']

In [42]:
list_df_region = pd.merge(list_df, cbsa_to_region, how='left', on = ['cbsa'])

del list_df

In [43]:
hosp_table = pd.merge(list_df_region, apc_to_description, how='left', on=['apc'])

del list_df_region

In [44]:
hosp_table = hosp_table.dropna(subset=['hosp_avg'])

In [45]:
phys_apc_data = phys_apc_data.dropna(subset=['cbsa'])

In [46]:
phys_apc_data['cbsa'] = phys_apc_data['cbsa'].astype('int')

In [47]:
phys_apc_data['apc'] = phys_apc_data['apc'].astype('int')

In [48]:
hosp_table

cbsa   apc  hosp_max  hosp_avg  hosp_min               region  \
0      20020  5072   1102.92   1028.73    986.77          AL - Dothan   
1      20020  5073   1916.44   1798.13   1685.36          AL - Dothan   
2      20020  5091   2202.57   2103.55   1994.50          AL - Dothan   
3      20020  5092   3876.45   3806.80   3737.14          AL - Dothan   
4      20020  5112   1078.61   1054.03   1029.46          AL - Dothan   
...      ...   ...       ...       ...       ...                  ...   
48974  21740  5302   1406.78   1406.78   1406.78  UT - Salt Lake City   
48979  21740  5361   4425.31   4425.31   4425.31  UT - Salt Lake City   
48988  21740  5431   1560.11   1560.11   1560.11  UT - Salt Lake City   
48991  21740  5491   1922.67   1922.67   1922.67  UT - Salt Lake City   
48993  21740  8011   2342.36   2342.36   2342.36  UT - Salt Lake City   

                                             description  
0        Level 2 Excision/ Biopsy/ Incision and Drainage  
1        Level 3 Excision/ Biopsy/ Incision and Drainage  
2      Level 1 Breast/Lymphatic Surgery and Related P...  
3      Level 2 Breast/Lymphatic Surgery and Related P...  
4                     Level 2 Musculoskeletal Procedures  
...                                                  ...  
48974                        Level 2 Upper GI Procedures  
48979           Level 1 Laparoscopy and Related Services  
48988                           Level 1 Nerve Procedures  
48991                     Level 1 Intraocular Procedures  
48993                 Comprehensive Observation Services  

[23354 rows x 7 columns]

In [49]:
# Getting all the unique apc's that are in the hospital table

hosp_apc_list = hosp_table['apc'].unique()

In [50]:
# Narrowing the Ambulatory Surgical Center table down to rows that have an apc that is in the hosp table

phys_apc_data = phys_apc_data[phys_apc_data['apc'].isin(hosp_apc_list)]

In [51]:
phys_apc_data = phys_apc_data[phys_apc_data['cbsa'].isin(hosp_table['cbsa'].unique())]

In [53]:
hcpcs_to_description = phys_apc_data.loc[:, ['hcpcs', 'description']]

In [55]:
hcpcs_to_description = hcpcs_to_description.drop_duplicates(subset=['hcpcs'])

In [58]:
hosp_table['state'] = hosp_table['region'].str[:2]

In [61]:
hosp_table

cbsa   apc  hosp_max  hosp_avg  hosp_min               region  \
0      20020  5072   1102.92   1028.73    986.77          AL - Dothan   
1      20020  5073   1916.44   1798.13   1685.36          AL - Dothan   
2      20020  5091   2202.57   2103.55   1994.50          AL - Dothan   
3      20020  5092   3876.45   3806.80   3737.14          AL - Dothan   
4      20020  5112   1078.61   1054.03   1029.46          AL - Dothan   
...      ...   ...       ...       ...       ...                  ...   
48974  21740  5302   1406.78   1406.78   1406.78  UT - Salt Lake City   
48979  21740  5361   4425.31   4425.31   4425.31  UT - Salt Lake City   
48988  21740  5431   1560.11   1560.11   1560.11  UT - Salt Lake City   
48991  21740  5491   1922.67   1922.67   1922.67  UT - Salt Lake City   
48993  21740  8011   2342.36   2342.36   2342.36  UT - Salt Lake City   

                                             description state  
0        Level 2 Excision/ Biopsy/ Incision and Drainage    AL  
1        Level 3 Excision/ Biopsy/ Incision and Drainage    AL  
2      Level 1 Breast/Lymphatic Surgery and Related P...    AL  
3      Level 2 Breast/Lymphatic Surgery and Related P...    AL  
4                     Level 2 Musculoskeletal Procedures    AL  
...                                                  ...   ...  
48974                        Level 2 Upper GI Procedures    UT  
48979           Level 1 Laparoscopy and Related Services    UT  
48988                           Level 1 Nerve Procedures    UT  
48991                     Level 1 Intraocular Procedures    UT  
48993                 Comprehensive Observation Services    UT  

[23354 rows x 8 columns]

In [62]:
cbsa_to_state = hosp_table.loc[:, ['cbsa', 'state']]

In [64]:
cbsa_to_state = cbsa_to_state.drop_duplicates(subset = ['cbsa'])

In [67]:
hosp_table['region'] = hosp_table['region'].str.split('-').str[1]

In [71]:
hosp_table['region'] = hosp_table['region'].str.strip()

In [72]:
hosp_table

cbsa   apc  hosp_max  hosp_avg  hosp_min          region  \
0      20020  5072   1102.92   1028.73    986.77          Dothan   
1      20020  5073   1916.44   1798.13   1685.36          Dothan   
2      20020  5091   2202.57   2103.55   1994.50          Dothan   
3      20020  5092   3876.45   3806.80   3737.14          Dothan   
4      20020  5112   1078.61   1054.03   1029.46          Dothan   
...      ...   ...       ...       ...       ...             ...   
48974  21740  5302   1406.78   1406.78   1406.78  Salt Lake City   
48979  21740  5361   4425.31   4425.31   4425.31  Salt Lake City   
48988  21740  5431   1560.11   1560.11   1560.11  Salt Lake City   
48991  21740  5491   1922.67   1922.67   1922.67  Salt Lake City   
48993  21740  8011   2342.36   2342.36   2342.36  Salt Lake City   

                                             description state  
0        Level 2 Excision/ Biopsy/ Incision and Drainage    AL  
1        Level 3 Excision/ Biopsy/ Incision and Drainage    AL  
2      Level 1 Breast/Lymphatic Surgery and Related P...    AL  
3      Level 2 Breast/Lymphatic Surgery and Related P...    AL  
4                     Level 2 Musculoskeletal Procedures    AL  
...                                                  ...   ...  
48974                        Level 2 Upper GI Procedures    UT  
48979           Level 1 Laparoscopy and Related Services    UT  
48988                           Level 1 Nerve Procedures    UT  
48991                     Level 1 Intraocular Procedures    UT  
48993                 Comprehensive Observation Services    UT  

[23354 rows x 8 columns]

In [73]:
phys_apc_data

npi                                    last_org_name entity  \
0      1003015843      NORTHBAY PHYSICIAN'S SURGERY CENTER, L.L.C.      O   
1      1003855594                        OHIO EYE ASSOCIATES, INC.      O   
2      1003861998    PHYSICIANS SURGERY CENTER OF CHATTANOOGA, LLC      O   
3      1003993965              EYE CONSULTANTS OF ATLANTA PIEDMONT      O   
4      1013096700                              CLEARER VISION, LLC      O   
...           ...                                              ...    ...   
61884  1952364499  DENTON REGIONAL AMBULATORY SURGERY CENTER, L.P.      O   
61885  1962869826                                    ENDOTECH, LLC      O   
61886  1962869826                                    ENDOTECH, LLC      O   
61887  1962869826                                    ENDOTECH, LLC      O   
61890  1992752224                           SPRINGFIELD CLINIC LLP      O   

                      street1    street2         city    zip state country  \
0          1006 NUT TREE ROAD        NaN    VACAVILLE  95687    CA      US   
1            466 S TRIMBLE RD        NaN    MANSFIELD  44906    OH      US   
2       924 SPRING CREEK ROAD        NaN  CHATTANOOGA  37412    TN      US   
3      3193 HOWELL MILL RD NW  SUITE 100      ATLANTA  30327    GA      US   
4                1059 NEAL ST    SUITE B   COOKEVILLE  38501    TN      US   
...                       ...        ...          ...    ...   ...     ...   
61884      3316 COLORADO BLVD        NaN       DENTON  76210    TX      US   
61885  8841 E BELL RD STE 101        NaN   SCOTTSDALE  85260    AZ      US   
61886  8841 E BELL RD STE 101        NaN   SCOTTSDALE  85260    AZ      US   
61887  8841 E BELL RD STE 101        NaN   SCOTTSDALE  85260    AZ      US   
61890           1025 S 6TH ST        NaN  SPRINGFIELD  62703    IL      US   

                    provider_type service_loc  hcpcs  \
0      Ambulatory Surgical Center           F  0191T   
1      Ambulatory Surgical Center           F  0191T   
2      Ambulatory Surgical Center           F  0191T   
3      Ambulatory Surgical Center           F  0191T   
4      Ambulatory Surgical Center           F  0191T   
...                           ...         ...    ...   
61884  Ambulatory Surgical Center           F  60240   
61885  Ambulatory Surgical Center           F  25075   
61886  Ambulatory Surgical Center           F  29845   
61887  Ambulatory Surgical Center           F  64910   
61890  Ambulatory Surgical Center           F  32550   

                                             description  services_cnt  \
0        Internal insertion of eye fluid drainage device          13.0   
1        Internal insertion of eye fluid drainage device          36.0   
2        Internal insertion of eye fluid drainage device          28.0   
3        Internal insertion of eye fluid drainage device          35.0   
4        Internal insertion of eye fluid drainage device          31.0   
...                                                  ...           ...   
61884                                 Removal of thyroid          19.0   
61885  Removal (less than 3 centimeters) tissue growt...          14.0   
61886   Removal of wrist joint lining using an endoscope          24.0   
61887                         Repair of nerve with graft         111.0   
61890  Insertion of permanent catheter for drainage o...          11.0   

       benefic_cnt  benefic_d_cnt  avg_medi_allowed_amt   cbsa  \
0               11             13           3042.237692  46700   
1               22             36           2320.990000  31900   
2               17             28           2408.890000  16860   
3               24             35           2432.058286  12060   
4               20             31           2229.990000  18260   
...            ...            ...                   ...    ...   
61884           19             19           2029.330000  19100   
61885           14             14            393.895000  38060

In [85]:
def phys_state(state_list):
    """This function takes a list of states as state codes and narrows the physcian data set to just those states
    it then iterates over the unique CBSA and HCPCS combos in that state and finds the medicare max/avg/min and makes
    those into a table."""
    state_phys_data = phys_apc_data[phys_apc_data['state'].isin(state_list)]
    list = []    
    for cbsa in state_phys_data['cbsa'].unique():
        for hcpcs in state_phys_data['hcpcs'].unique():
            selection = state_phys_data[(state_phys_data['cbsa'] == cbsa) & (state_phys_data['hcpcs'] == hcpcs)]
            max_value = round(selection.avg_medi_allowed_amt.max(), 2)
            avg_value = round(selection.avg_medi_allowed_amt.mean(), 2)
            min_value = round(selection.avg_medi_allowed_amt.min(), 2)
            list.append([cbsa, hcpcs, max_value, avg_value, min_value])
    list = pd.DataFrame(list)
    list.columns = ['cbsa','hcpcs', 'office_max', 'office_avg', 'office_min']
    global phys_data_state 
    phys_data_state = list.dropna(subset=['office_avg'])
    return phys_data_state

In [107]:
phys_state(['TN'])

cbsa  hcpcs  office_max  office_avg  office_min
0     16860  0191T     2408.89     2396.15     2370.66
1     16860  29823      601.13      601.13      601.13
2     16860  29824      626.68      626.68      626.68
3     16860  29827     1891.32     1891.32     1891.32
4     16860  29880     1069.53     1067.85     1066.18
...     ...    ...         ...         ...         ...
3039  46100  67904      562.85      562.85      562.85
3044  46100  49505     1254.94     1254.94     1254.94
3077  46100  47563     1759.74     1759.74     1759.74
3166  46100  67950      544.24      544.24      544.24
3347  46460  43453      524.92      524.92      524.92

[493 rows x 5 columns]

In [108]:
phys_data_state = pd.merge(phys_data_state, hcpcs_to_apc, how = 'left', on = ['hcpcs'])

In [109]:
merged_data = pd.merge(phys_data_state, hosp_table, how = 'left', on = ['cbsa', 'apc'])

In [112]:
merged_data = merged_data[merged_data['state'] == 'TN']

In [113]:
merged_data.to_csv('../data/file_for_viz.csv')

In [117]:
merged_data

cbsa  hcpcs  office_max  office_avg  office_min  \
0    16860  0191T     2408.89     2396.15     2370.66   
1    16860  29823      601.13      601.13      601.13   
2    16860  29824      626.68      626.68      626.68   
3    16860  29827     1891.32     1891.32     1891.32   
4    16860  29880     1069.53     1067.85     1066.18   
..     ...    ...         ...         ...         ...   
488  46100  67904      562.85      562.85      562.85   
489  46100  49505     1254.94     1254.94     1254.94   
490  46100  47563     1759.74     1759.74     1759.74   
491  46100  67950      544.24      544.24      544.24   
492  46460  43453      524.92      524.92      524.92   

                       descriptor     apc  hosp_max  hosp_avg  hosp_min  \
0    Insert ant segment drain int  5492.0   3148.69   2858.93   2397.92   
1    Shoulder arthroscopy/surgery  5113.0   2245.72   2158.94   2076.43   
2    Shoulder arthroscopy/surgery  5113.0   2245.72   2158.94   2076.43   
3    Arthroscop rotator cuff repr  5114.0   4809.09   4652.22   4569.48   
4        Knee arthroscopy/surgery  5113.0   2245.72   2158.94   2076.43   
..                            ...     ...       ...       ...       ...   
488          Repair eyelid defect  5503.0   1567.07   1567.07   1567.07   
489   Prp i/hern init reduc >5 yr  5341.0   2565.27   2500.27   2384.42   
490  Laparo cholecystectomy/graph  5361.0   3867.42   3553.21   3244.20   
491            Revision of eyelid  5503.0   1567.07   1567.07   1567.07   
492              Dilate esophagus  5302.0   1146.71   1122.39   1098.08   

          region                                        description state  
0    Chattanooga                     Level 2 Intraocular Procedures    TN  
1    Chattanooga                 Level 3 Musculoskeletal Procedures    TN  
2    Chattanooga                 Level 3 Musculoskeletal Procedures    TN  
3    Chattanooga                 Level 4 Musculoskeletal Procedures    TN  
4    Chattanooga                 Level 3 Musculoskeletal Procedures    TN  
..           ...                                                ...   ...  
488    Nashville  Level 3 Extraocular, Repair, and Plastic Eye P...    TN  
489    Nashville  Abdominal/Peritoneal/Biliary and Related Proce...    TN  
490    Nashville           Level 1 Laparoscopy and Related Services    TN  
491    Nashville  Level 3 Extraocular, Repair, and Plastic Eye P...    TN  
492      Jackson                        Level 2 Upper GI Procedures    TN  

[419 rows x 13 columns]

In [96]:
phys_data_state

cbsa  hcpcs  office_max  office_avg  office_min  \
0    16860  0191T     2408.89     2396.15     2370.66   
1    16860  29823      601.13      601.13      601.13   
2    16860  29824      626.68      626.68      626.68   
3    16860  29827     1891.32     1891.32     1891.32   
4    16860  29880     1069.53     1067.85     1066.18   
..     ...    ...         ...         ...         ...   
488  46100  67904      562.85      562.85      562.85   
489  46100  49505     1254.94     1254.94     1254.94   
490  46100  47563     1759.74     1759.74     1759.74   
491  46100  67950      544.24      544.24      544.24   
492  46460  43453      524.92      524.92      524.92   

                       descriptor     apc  
0    Insert ant segment drain int  5492.0  
1    Shoulder arthroscopy/surgery  5113.0  
2    Shoulder arthroscopy/surgery  5113.0  
3    Arthroscop rotator cuff repr  5114.0  
4        Knee arthroscopy/surgery  5113.0  
..                            ...     ...  
488          Repair eyelid defect  5503.0  
489   Prp i/hern init reduc >5 yr  5341.0  
490  Laparo cholecystectomy/graph  5361.0  
491            Revision of eyelid  5503.0  
492              Dilate esophagus  5302.0  

[493 rows x 7 columns]

In [94]:
list2 = []

for cbsa in phys_testing['cbsa'].unique():
    for hcpcs in phys_testing['hcpcs'].unique():
        selection = phys_testing[(phys_testing['cbsa'] == cbsa) & (phys_testing['hcpcs'] == hcpcs)]
        max_value = round(selection.avg_medi_allowed_amt.max(), 2)
        avg_value = round(selection.avg_medi_allowed_amt.mean(), 2)
        min_value = round(selection.avg_medi_allowed_amt.min(), 2)
        list2.append([cbsa, hcpcs, max_value, avg_value, min_value])

KeyboardInterrupt: 

In [82]:
list2_df = pd.DataFrame(list2)

del list2

In [87]:
list2_df.columns = ['cbsa','hcpcs', 'office_max', 'office_avg', 'office_min']

In [88]:
list2_df

cbsa  hcpcs  office_max  office_avg  office_min
0     46700  0191T     3549.09     3295.66     3042.24
1     46700  20610       35.35       30.81       26.26
2     46700  29823      837.13      837.13      837.13
3     46700  29824     1016.51     1016.51     1016.51
4     31900  0191T     2320.99     2320.99     2320.99
...     ...    ...         ...         ...         ...
1627  32900  29824      810.97      810.97      810.97
1631  16580  29824      720.14      720.14      720.14
1635  11620  29824      827.61      827.61      827.61
1639  24940  29824      655.09      655.09      655.09
1643  47380  29824      742.05      742.05      742.05

[897 rows x 5 columns]

In [85]:
list2_df = list2_df.dropna(subset=['office_avg'])

In [266]:
hosp_office_data = pd.merge(list2_df, hosp_table, how='inner', on = ['cbsa', 'apc'])

del list2_df
del hosp_table
del phys_apc_data

In [268]:
hosp_office_data

cbsa   apc  office_max  office_avg  office_min  hosp_max  hosp_avg  \
0     27740  5113     1050.94      772.29      597.59   1906.12   1906.12   
1     27740  5114     2211.67     2211.67     2211.67   4285.93   4207.59   
2     27740  5464    19140.93    19140.93    19140.93  21335.92  21335.92   
3     27740  5491      837.80      825.64      812.00   1497.46   1497.46   
4     27740  5073      889.35      889.35      889.35   1764.02   1764.02   
...     ...   ...         ...         ...         ...       ...       ...   
4845  21820  5491     1010.83     1005.72     1000.62   2212.06   2212.06   
4846  34260  5113     1052.65      852.70      596.51   2275.25   2275.25   
4847  34260  5114     2288.29     2183.22     2078.14   4824.46   4824.46   
4848  34260  5431      668.70      644.60      620.49   1454.97   1454.97   
4849  34260  5112      500.54      439.37      378.19   1085.76   1085.76   

      hosp_min             region  \
0      1906.12  TN - Johnson City   
1      4129.25  TN - Johnson City   
2     21335.92  TN - Johnson City   
3      1497.46  TN - Johnson City   
4      1764.02  TN - Johnson City   
...        ...                ...   
4845   2212.06     AK - Anchorage   
4846   2275.25   MO - Springfield   
4847   4824.46   MO - Springfield   
4848   1454.97   MO - Springfield   
4849   1085.76   MO - Springfield   

                                          description  
0                  Level 3 Musculoskeletal Procedures  
1                  Level 4 Musculoskeletal Procedures  
2      Level 4 Neurostimulator and Related Procedures  
3                      Level 1 Intraocular Procedures  
4     Level 3 Excision/ Biopsy/ Incision and Drainage  
...                                               ...  
4845                   Level 1 Intraocular Procedures  
4846               Level 3 Musculoskeletal Procedures  
4847               Level 4 Musculoskeletal Procedures  
4848                         Level 1 Nerve Procedures  
4849               Level 2 Musculoskeletal Procedures  

[4850 rows x 10 columns]

In [96]:
# Selcting a cbsa and a apc and returing the max, mean, and min values for those selections.

selection = hosp_payments_cbsa[(hosp_payments_cbsa['cbsa'] == 20020) & (hosp_payments_cbsa['apc'] == 5072)]
max_value = round(selection.avg_medi_allowed_amt.max(), 2)
avg_value = round(selection.avg_medi_allowed_amt.mean(), 2)
min_value = round(selection.avg_medi_allowed_amt.min(), 2)

print('Region = ', selection['region'][0])
print('Procedure = ', selection['description'][0])
print()
print('Max = ', max_value)
print('Mean = ', avg_value)
print('Min = ', min_value)

Region =  AL - Dothan
Procedure =  Level 2 Excision/ Biopsy/ Incision and Drainage

Max =  1102.92
Mean =  1028.73
Min =  986.77
